# Set Cover problem

See: https://en.wikipedia.org/wiki/Set_cover_problem

In [997]:
from random import random, seed
import math
from itertools import accumulate
import numpy as np
import matplotlib.pyplot as plt



from icecream import ic

## Reproducible Initialization

If you want to get reproducible results, use `rng` (and restart the kernel); for non-reproducible ones, use `np.random`.

In [998]:
UNIVERSE_SIZE = 100000
NUM_SETS = 10000
DENSITY = 0.1 #how dense are the sets, how many elements are covered by each set


rng = np.random.Generator(np.random.PCG64([UNIVERSE_SIZE, NUM_SETS, int(10_000 * DENSITY)]))

In [999]:
# DON'T EDIT THESE LINES!

SETS = np.random.random((NUM_SETS, UNIVERSE_SIZE)) < DENSITY
for s in range(UNIVERSE_SIZE):
    if not np.any(SETS[:, s]):
        SETS[np.random.randint(NUM_SETS), s] = True
COSTS = np.pow(SETS.sum(axis=1), 1.1)

## Helper Functions

In [1000]:
def valid(solution):
    """Checks wether solution is valid (ie. covers all universe)"""
    return np.all(np.logical_or.reduce(SETS[solution]))


def cost(solution):
    """Returns the cost of a solution (to be minimized)"""
    return COSTS[solution].sum()


## CHOSEN SOLUTION

## Fitness
The fitness defines the goodness of a solution. I decided to return a tuple containing (coverage,-cost).
The coverage represents the number of elements in the universe that are covered, while the cost is defined as above.

In [1001]:
#The fitness function evaluate the goodness of a solution
def fitness(solution: np.ndarray):
    """Returns a tuple containing first the number of elements covered by the solution and then the cost of the solution (negative->to be maximized)"""
    return (np.sum(np.logical_or.reduce(SETS[solution])), -cost(solution))

## GREEDY INITIALIZATION

In [1002]:
def greedy_cover_selection():
    """Greedy approach to generate an initial solution for the Set Cover problem."""
    covered_elements = np.zeros(SETS.shape[1], dtype=bool)
    selected_sets = np.zeros(len(SETS), dtype=bool)
    
    while not np.all(covered_elements):
        optimal_set = -1
        optimal_cost_ratio = float('inf')
        
        for idx in range(len(SETS)):
            if selected_sets[idx]:  # Skip already selected sets
                continue
            uncovered_elements = np.logical_and(SETS[idx], np.logical_not(covered_elements)).sum()
            if uncovered_elements > 0:
                cost_efficiency = COSTS[idx] / uncovered_elements
                if cost_efficiency < optimal_cost_ratio:
                    optimal_cost_ratio = cost_efficiency
                    optimal_set = idx
        
        selected_sets[optimal_set] = True
        covered_elements = np.logical_or(covered_elements, SETS[optimal_set])
        
    return selected_sets


In [1003]:
history = []
starting_solution = greedy_cover_selection()
solution_fitness = fitness(starting_solution)
ic(solution_fitness) #starting point
history.append(solution_fitness[1])


ic| solution_fitness: (np.int64(100000), np.float64(-1528012.1791879225))


## TWEAK FUNCTION

In [1004]:
def multiple_mutation_strength(solution: np.ndarray, strength: float)->np.ndarray:  
    new_solution = solution.copy()
    mask = rng.random(NUM_SETS) < strength
    new_solution = np.logical_xor(solution,mask)
    return new_solution

## PARAMETER TUNING
The algorithm should adapt to changes to instances of the problem
If we use a modified version of iterated local search, we have basically to adapt 2 main parameters:
- Strength (approximately the percentage of elements tweaked with respect to the current solution)
- Number of iterations (function of the problem size: it should increase with it, but it depends on the time required to execute the algorithm)
  
The strength depends on the density: if the density is high, the solution is less sensitive to small changes.

In [1005]:
BASE_ITERATIONS = 20
number_iteration = int(BASE_ITERATIONS*math.log(UNIVERSE_SIZE))/2
number_iteration_exploration = int(number_iteration*0.2)
number_iteration_exploitation = int(number_iteration - number_iteration_exploration)
ic(number_iteration, number_iteration_exploration, number_iteration_exploitation)

#At the beginning I should favor exploration, then I should favor exploitation
#The strength depends on the density: if the density is high, the solution is less sensitive to small changes
BASE_STRENGTH = (0.1/UNIVERSE_SIZE)
strength = BASE_STRENGTH * (1+DENSITY)

ic| number_iteration: 115.0
    number_iteration_exploration: 23
    number_iteration_exploitation: 92


## Hill climbing with self adaptive strength
The starting point is the empty solution. There are two cicles: the outer one where the tweak uses an higher strength to favor exploration and an inner one, where the strength is lower, to favor exploitation.
The strength is modified based on the convergence of the solution: when we are close to all the elements covered, it is decreased not to add too many sets but still reach a valid solution. When we are far from a valid solution, we increase it, so that, statistically, we are able to tweak more elements and reach an higher coverage (particularly at the beginning). In case the universe is completely covered, we have iterations left to improve the current solution: so we try to increase or decrease the temperature to favor exploration/exploitation based on the previous results.

In [1006]:

best = np.full(NUM_SETS,False) #Completely empty solution -> no set selected
count_iteration = 0
buffer = []
BUF_LENGTH = int(number_iteration/5)
solution_fitness = (0,-0)
for steps_perturbation in range(int(number_iteration_exploration)):
    if(solution_fitness[0]<UNIVERSE_SIZE/1.5):
        strength*=(1+(1-DENSITY)*5)
    elif(solution_fitness[0]>UNIVERSE_SIZE/1.5 and solution_fitness[0]<UNIVERSE_SIZE):
        if(strength>1*DENSITY):
            strength*=1/(DENSITY*15)
    else:
        buffer = buffer[BUF_LENGTH:]
        if(np.sum(buffer)>1):
            strength *= (1+DENSITY)
        else:
            strength *= (1-DENSITY)
    solution_perturbated = multiple_mutation_strength(best,strength*4) #a huge mutation->exploration
    for steps_local in range(number_iteration_exploitation):
        count_iteration+=1
        new_solution = multiple_mutation_strength(solution_perturbated,strength) 
        fitness_new = fitness(new_solution)
        buffer.append(fitness_new > solution_fitness)
        if fitness_new > solution_fitness:
            solution_fitness = fitness_new
            solution_perturbated = new_solution
            best = new_solution.copy()
ic(count_iteration)
ic(fitness(best))
ic(valid(best))

ic| count_iteration: 2116
ic| fitness(best): (np.int64(100000), np.float64(-2437831.577057764))
ic| valid(best): np.True_


np.True_

## Starting with the Greedy solution
Taking the greedy solution as a starting point is obviously the best option, even if my algorithm is not able to find a solution that improves the starting point. Thus, I leave the previous section as a demonstration that the algorithm works. Basically it's just an Hill Climbing with self-adaptive temperature (based on the density) if the starting point is already valid. But, since the greedy starting point is really very optimized, it is difficult to find something better.

In [1007]:
BASE_ITERATIONS = 20
number_iteration = int(BASE_ITERATIONS*math.log(UNIVERSE_SIZE))/2
number_iteration_exploration = int(number_iteration*0.2)
number_iteration_exploitation = int(number_iteration - number_iteration_exploration)
ic(number_iteration, number_iteration_exploration, number_iteration_exploitation)

#At the beginning I should favor exploration, then I should favor exploitation
#The strength depends on the density: if the density is high, the solution is less sensitive to small changes
BASE_STRENGTH = (0.1/UNIVERSE_SIZE)
strength = BASE_STRENGTH * (1+DENSITY)

ic| number_iteration: 115.0
    number_iteration_exploration: 23
    number_iteration_exploitation: 92


In [1008]:

best = starting_solution.copy()
count_iteration = 0
buffer = []
BUF_LENGTH = int(number_iteration/5)
solution_fitness = (0,-0)
for steps_perturbation in range(int(number_iteration_exploration)):
    if(solution_fitness[0]<UNIVERSE_SIZE/1.5):
        strength*=(1+(1-DENSITY)*5)
    elif(solution_fitness[0]>UNIVERSE_SIZE/1.5 and solution_fitness[0]<UNIVERSE_SIZE):
        if(strength>1*DENSITY):
            strength*=1/(DENSITY*15)
    else:
        buffer = buffer[BUF_LENGTH:]
        if(np.sum(buffer)>1):
            strength *= (1+DENSITY)
        else:
            strength *= (1-DENSITY)

    solution_perturbated = multiple_mutation_strength(best,strength*4) #a huge mutation->exploration
    for steps_local in range(number_iteration_exploitation):
        count_iteration+=1
        new_solution = multiple_mutation_strength(solution_perturbated,strength) 
        fitness_new = fitness(new_solution)
        buffer.append(fitness_new > solution_fitness)
        if fitness_new > solution_fitness:
            solution_fitness = fitness_new
            solution_perturbated = new_solution
            best = new_solution.copy()
ic(count_iteration)
ic(fitness(best))
ic(valid(best))


ic| count_iteration: 2116
ic| fitness(best): (np.int64(100000), np.float64(-1528012.1791879225))
ic| valid(best): np.True_


np.True_